In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


In [2]:
## read the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Check if columns exist before dropping
columns_to_drop = ['RowNumber', 'CustomerId', 'Surname']
existing_columns_to_drop = [col for col in columns_to_drop if col in data.columns]
data.drop(existing_columns_to_drop, axis=1, inplace=True)
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## geography and gender data is categorical, so we need to encode it
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data['Gender'].head()

0    0
1    0
2    0
3    0
4    0
Name: Gender, dtype: int64

In [5]:
## we need to onehot encode the geography column becase it has more than 2 categories and we don't want the model to think that one category is better than the other
from sklearn.preprocessing import OneHotEncoder
one_shot_encoder = OneHotEncoder()
geo_encoder = one_shot_encoder.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
one_shot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=one_shot_encoder.get_feature_names_out(['Geography']))


In [8]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [9]:
## combine one hot encoded geography with the original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## save the encoders and scaler
with open('labe_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('one_shot_encoder.pkl', 'wb') as f:
    pickle.dump(one_shot_encoder, f)

In [11]:
## divide the data set into dependant and independant variables
X = data.drop('Exited', axis=1)
y = data['Exited']

## split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [12]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

2024-10-24 16:41:47.087262: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 16:41:47.095267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 16:41:47.103766: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 16:41:47.106225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 16:41:47.113088: I tensorflow/core/platform/cpu_feature_guar

In [15]:
X_train.shape[1]


12

In [19]:
## Build Model

## start with input layer with x.shape
model = Sequential([
    Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(units=32, activation='relu'),
    Dense(units=1, activation='sigmoid')
])


/home/onur/miniconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [21]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [25]:
## setup tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [29]:
## set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [30]:
## training the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[tensorflow_callback, early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - accuracy: 0.8669 - loss: 0.3239 - val_accuracy: 0.8635 - val_loss: 0.3424
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - accuracy: 0.8757 - loss: 0.3040 - val_accuracy: 0.8625 - val_loss: 0.3506
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - accuracy: 0.8652 - loss: 0.3186 - val_accuracy: 0.8580 - val_loss: 0.3533
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - accuracy: 0.8729 - loss: 0.3113 - val_accuracy: 0.8615 - val_loss: 0.3497
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.8688 - loss: 0.3154 - val_accuracy: 0.8600 - val_loss: 0.3527
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - accuracy: 0.8721 - loss: 0.3087 - val_accuracy: 0.8605 - val_loss: 0.3491
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - accuracy: 0.8665 - loss: 0.3127 - val_accuracy: 0.8605 - val_loss: 0.3412
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - accuracy: 0.8725 - loss: 0

In [31]:
model.save('model.h5')

In [32]:
## load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
## load the pickle file
